In [ ]:
import os
import json
from pathlib import Path

from parso.python.tree import Literal

benchmark_path = Path("../target/criterion")
maps_paths = list(map(lambda p: benchmark_path / Path(p), filter(lambda n: n != "report", os.listdir(benchmark_path))))
os.listdir()

methods = ["fast_sssp_sequential", "dijkstra", "dijkstra_fibonacci"]


# {map_name: {method_name: {metric_name: value}}}
def load_json(filename: Path) -> dict:
    with open(filename) as f:
        return json.load(f)


data = {
    m.name: {method: load_json(m / Path(method) / Path("new/estimates.json")) for method in methods}
    for m in maps_paths
}
data


In [ ]:
from typing import Any

pairs = [
    ("jan_mayen", 500),
    ("gibraltar", 500),
    ("monaco", 500),
    ("san_marino", 100),
    ("andorra", 100),
    ("gotland", 100),
    ("malta", 25),
    ("reykjavik", 25),
    ("budapest", 25),
    ("luxembourg", 25),
    ("haiti", 20),
    ("iceland", 10),
    ("stockholm", 10),
    ("missisippi", 10),
    ("peru", 5),
    ("sweden", 5),
]


def normalize_recursive(item: Any, pair_count: int) -> Any:
    if isinstance(item, dict):
        return {k: (normalize_recursive(v, pair_count) if k != "confidence_level" else v) for k, v in item.items()}
    if isinstance(item, list):
        return [normalize_recursive(i, pair_count) for i in item]
    if isinstance(item, (float, int)):
        return item / pair_count
    return item


normalized_data = {area: normalize_recursive(data[area + ".osm"], pair_count) for area, pair_count in pairs}
normalized_data


In [ ]:
graph_data = {
    "jan_mayen": {"edges": 29_786, "nodes": 13_230},
    "monaco": {"edges": 72_318, "nodes": 32_492},
    "gibraltar": {"edges": 100_284, "nodes": 44_639},
    "san_marino": {"edges": 341_976, "nodes": 154_249},
    "andorra": {"edges": 1043_844, "nodes": 449_273},
    "gotland": {"edges": 1634_818, "nodes": 725_852},
    "malta": {"edges": 1_818_642, "nodes": 734_962},

    "reykjavik": {"edges": 2_241_396, "nodes": 1_051_160},
    "budapest": {"edges": 5_187_404, "nodes": 2_443_154},
    "luxembourg": {"edges": 10_664_130, "nodes": 3_916_210},
    "haiti": {"edges": 17_186_498, "nodes": 8_497_106},

    "iceland": {"edges": 21_055_604, "nodes": 10_350_896},
    "missisippi": {"edges": 21_372_374, "nodes": 10_464_418},
    "stockholm": {"edges": 16_459_816, "nodes": 73_07_104},
    "peru": {"edges": 76_084_784, "nodes": 37_194_932},
    "sweden": {"edges": 220_005_662, "nodes": 98_808_100}, }
normalized_data_with_graph = {
    area: {"methods": normalized_data[area], "graph": graph_data[area]} for area, v in normalized_data.items()
}
normalized_data_with_graph["jan_mayen"]["graph"]


In [ ]:
# Optional: restrict the amount of data

# Split the data by graph size (nodes); <= 1_000_000 nodes, between 1_000_000 and 10_000_000 nodes, and > 10_000_000 nodes
small_max = 1_000_000
medium_max = 11_000_000

split_graph_data = {
    'small': {k: v for k, v in normalized_data_with_graph.items() if v["graph"]["nodes"] <= small_max},
    'medium': {k: v for k, v in normalized_data_with_graph.items() if small_max < v["graph"]["nodes"] <= medium_max},
    'large': {k: v for k, v in normalized_data_with_graph.items() if v["graph"]["nodes"] > medium_max},
}

filtered_methods = [m for m in methods if m != "dijkstra_fibonacci"]


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
colors = {"dijkstra": "C0", "dijkstra_fibonacci": "C1", "fast_sssp_sequential": "C2"}
sm_data = {**split_graph_data['small'], **split_graph_data['medium']}



In [ ]:
from matplotlib import ticker
# The method graphs
from scipy.optimize import curve_fit

mp = {
    'dijkstra': lambda x, a, b: a * 2 * x * np.log(x) + b,
    'dijkstra_fibonacci': lambda x, a, b: x + a * x * np.log(x) ** 2 / 3 + b,
    'fast_sssp_sequential': lambda x, a, b: a * x * np.log(x) ** 2 / 3 + b,
}

this_data = sm_data
for method in methods:
    plt.figure(figsize=(9, 6))
    x = np.array([normalized_data_with_graph[a]["graph"]["nodes"] for a in this_data])
    y = np.array([normalized_data_with_graph[a]["methods"][method]["mean"]["point_estimate"] for a in
                  this_data])
    z = mp[method]

    fitted_params, _ = curve_fit(z, x, y)

    # Generate points for the trendline
    x_sorted = np.sort(x)
    y_fitted = z(x_sorted, *fitted_params)

    plt.plot(x_sorted, y_fitted, label=f"Trendline, fitted to O(Nlog(N)2/3", c=colors.get(method, None))
    plt.scatter(x, y, label=method, s=30, alpha=0.8, c=colors.get(method, None))
    plt.xlabel("Number of nodes")
    plt.ylabel("Mean time (s)")
    plt.title("Mean runtime vs Number of nodes")
    # plt.legend(title="Legend")
    plt.gca().xaxis.set_major_formatter(
        ticker.FuncFormatter(
            lambda x, p: f'{x / 1e6:.0f}M' if x >= 1e6 else f'{x / 1e3:.0f}k' if x >= 1e3 else f'{x:.0f}'))
    plt.gca().yaxis.set_major_formatter(
        ticker.FuncFormatter(
            lambda x, p: f'{x / 1e6:.0f}' if x >= 1e6 else f'{x / 1e3:.0f}' if x >= 1e3 else f'{x:.0f}'))

    plt.legend
    plt.tight_layout()
    plt.savefig(f"graphs/{method}.png")
    plt.show()

In [ ]:
# Mean dijkstra vs fast graph
plt.figure(figsize=(9, 6))
for method in filtered_methods:
    x = [normalized_data_with_graph[a]["graph"]["nodes"] for a in sm_data]
    y = [normalized_data_with_graph[a]["methods"][method]["mean"]["point_estimate"] for a in sm_data]
    plt.scatter(x, y, label=method, s=30, alpha=0.8, c=colors.get(method, None))

plt.xlabel("Number of nodes")
plt.ylabel("Mean time (s)")
plt.title("Mean point_estimate vs Number of nodes by method")
plt.legend(title="Method")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

areas = []
speedups = []
labels = []

for area, data in normalized_data_with_graph.items():
    dijkstra_time = data["methods"]["dijkstra"]["mean"]["point_estimate"]
    fast_sssp_time = data["methods"]["fast_sssp_sequential"]["mean"]["point_estimate"]
    speedup = dijkstra_time / fast_sssp_time

    # Format node count in human readable format
    nodes = data["graph"]["nodes"]
    if nodes >= 1_000_000:
        node_label = f"{nodes / 1_000_000:.1f}M"
    elif nodes >= 1_000:
        node_label = f"{nodes / 1_000:.0f}k"
    else:
        node_label = str(nodes)

    areas.append(area)
    speedups.append(speedup)
    labels.append(f"{area.replace('_', ' ').title()} ({node_label} nodes)")

# Sort by number of nodes for better visualization
sorted_indices = sorted(range(len(areas)), key=lambda i: normalized_data_with_graph[areas[i]]["graph"]["nodes"])
sorted_areas = [areas[i] for i in sorted_indices]
sorted_speedups = [speedups[i] for i in sorted_indices]
sorted_labels = [labels[i] for i in sorted_indices]

# Create the column graph
plt.figure(figsize=(14, 8))
bars = plt.bar(range(len(sorted_areas)), sorted_speedups, color=['green' if s > 1 else 'red' for s in sorted_speedups])

# Add speedup numbers above the bars
for i, (bar, speedup) in enumerate(zip(bars, sorted_speedups)):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2., height + 0.1,
             f'{speedup:.2f}x', ha='center', va='bottom', fontsize=10)

# Add a horizontal line at y=1 to show no speedup/slowdown
plt.axhline(y=1, color='black', linestyle='--', alpha=0.7, label='No speedup/slowdown')

plt.xlabel('Area')
plt.ylabel('Speedup (dijkstra / fast_sssp_sequential)')
plt.title('Speedup of fast_sssp_sequential compared to dijkstra')
plt.xticks(range(len(sorted_areas)), sorted_labels, rotation=30, ha='right')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# Print speedup values for reference
for i, (area, speedup) in enumerate(zip(sorted_areas, sorted_speedups)):
    print(f"{sorted_labels[i]}: {speedup:.2f}x {'speedup' if speedup > 1 else 'slowdown'}")
